<a href="https://colab.research.google.com/github/senadhivishwajith/-Cloud-Detection-and-Tracking/blob/main/convLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install tqdm

Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 122.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6.7 MB/s e

In [ ]:
import torch

if torch.cuda.is_available():
    print("✅ CUDA is available!")
    print(f"🖥️  GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"🧠 Memory Allocated: {torch.cuda.memory_allocated() / 1024 ** 2:.2f} MB")
    print(f"🔢 Device Count: {torch.cuda.device_count()}")
else:
    print("❌ CUDA is NOT available. Using CPU instead.")

✅ CUDA is available!
🖥️  GPU Name: Tesla T4
🧠 Memory Allocated: 0.00 MB
🔢 Device Count: 1


In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
%cd /content/drive/MyDrive/cloud_convlstm

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[Errno 2] No such file or directory: '/content/drive/MyDrive/cloud_convlstm'
/content


In [ ]:
import torch
from torch.utils.data import Dataset
import numpy as np
import os

class CloudSequenceDataset(Dataset):
    def __init__(self, npz_folder):
        self.file_list = [os.path.join(npz_folder, f) for f in os.listdir(npz_folder) if f.endswith('.npz')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        data = np.load(self.file_list[idx])
        x = data['X']  # (30, 128, 128)
        y = data['Y']  # (128, 128)
        opacity = data['opacity']  # (30,)

        x = torch.tensor(x, dtype=torch.float32).unsqueeze(1)  # (30, 1, 128, 128)
        y = torch.tensor(y, dtype=torch.float32).unsqueeze(0)  # (1, 128, 128)
        opacity = torch.tensor(opacity, dtype=torch.float32)

        return x, y, opacity

In [ ]:
import torch.nn as nn
import torch

class ConvLSTMCell(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size):
        super().__init__()
        self.hidden_dim = hidden_dim
        padding = kernel_size // 2
        self.conv = nn.Conv2d(input_dim + hidden_dim, 4 * hidden_dim, kernel_size, padding=padding)

    def forward(self, x, h_cur, c_cur):
        combined = torch.cat([x, h_cur], dim=1)
        conv_output = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.chunk(conv_output, 4, dim=1)

        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)
        return h_next, c_next

    def init_hidden(self, batch_size, image_size, device):
        H, W = image_size
        return (
            torch.zeros(batch_size, self.hidden_dim, H, W, device=device),
            torch.zeros(batch_size, self.hidden_dim, H, W, device=device)
        )

class ConvLSTM(nn.Module):
    def __init__(self, input_dim=1, hidden_dim=32, kernel_size=3):
        super().__init__()
        self.cell = ConvLSTMCell(input_dim, hidden_dim, kernel_size)
        self.output_conv = nn.Conv2d(hidden_dim, 1, kernel_size=1)

    def forward(self, input_seq):
        B, T, C, H, W = input_seq.size()
        h, c = self.cell.init_hidden(B, (H, W), input_seq.device)

        for t in range(T):
            h, c = self.cell(input_seq[:, t], h, c)

        out = self.output_conv(h)
        return torch.sigmoid(out)

In [ ]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm

def train(model, dataset, device, batch_size=16, epochs=15, lr=1e-3):
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    model.train()

    for epoch in range(1, epochs + 1):
        total_loss = 0
        loop = tqdm(dataloader, desc=f"Epoch {epoch}/{epochs}")
        for x, y, _ in loop:
            x, y = x.to(device), y.to(device)
            pred = model(x)
            loss = F.binary_cross_entropy(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            loop.set_postfix(loss=loss.item())

        print(f"✅ Epoch {epoch} Avg Loss: {total_loss / len(dataloader):.6f}")

In [ ]:
import os
import gc
import torch

# === Import garbage collector ===
gc.collect()
torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# === Paths and parameters ===
npz_folder = "/content/drive/MyDrive/Cloud convLSTM/data/cloud_tracks_npz"
batch_size = 16
epochs = 15
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# === Load dataset and model ===
dataset = CloudSequenceDataset(npz_folder)
model = ConvLSTM().to(device)

# === Train the model ===
train(model, dataset, device, batch_size=batch_size, epochs=epochs)

# === Save the model ===
os.makedirs("models", exist_ok=True)
torch.save(model.state_dict(), "models/convlstm_model.pth")
print("✅ Model saved to models/convlstm_model.pth")


Using device: cuda


NameError: name 'CloudSequenceDataset' is not defined